In [48]:
import math
import numpy as np
import pandas as pd
import pickle
import nltk

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as patches
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from gensim import corpora
import sddk

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

In [2]:
# for exporting data to googlesheets/sciencedata
# (feel free to skip)

s = sddk.cloudSession("sciencedata.dk")
# establish connection with gogglesheets...
file_data = s.read_file("https://sciencedata.dk/files/ServiceAccountsKey.json", "dict") # or load it from a local storage: json.load(open("../../ServiceAccountsKey.json", "r"))
credentials = service_account.Credentials.from_service_account_info(file_data)
gc = gspread.Client(auth=credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']))
paul_results = gc.open_by_url("https://docs.google.com/spreadsheets/d/1h4M-gK9TPIfeTV528tUuPBfZF1wtcNCA10yIlJYqGTE/edit?usp=sharing")

endpoint variable has been configured to: https://sciencedata.dk/files/


In [3]:
LAGTec = pd.read_json("../data/large_files/LAGTec.json")

In [4]:
len(LAGTec["author"].unique().tolist())

49

In [5]:
len(LAGTec)

148

In [6]:
LAGTec["wordcount"].sum()

3566823

In [28]:
LAGTec["lemmatized_sentences"].apply(lambda x: len([w for s in x for w in s])).sum()

1506066

In [7]:
LAGTec

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,n_sentences,lemmatized_sentences,sentences_paul,sentences_paul_N,sentences_apostle,sentences_apostle_N,paul_N
385,tlg0031.tlg001.perseus-grc2.xml,Gospel of Matthew,Gospel of Matthew,18288,tlg0031a,tlg0031.tlg001,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],ΒΙΒΛΟΣ γενέσεως Ἰησοῦ Χριστοῦ υἱοῦ Δαυεὶδ υἱο...,1276,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί...",[],0,"[[ἀπόστολος, ὄνομα, εἰμί]]",1,0
386,tlg0031.tlg002.perseus-grc2.xml,Gospel of Mark,Gospel of Mark,11274,tlg0031b,tlg0031.tlg002,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],ΑΡΧΗ τοῦ εὐαγγελίου Ἰησοῦ Χριστοῦ . Καθὼς γέγ...,790,"[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός], [γράφω, ...",[],0,"[[ποιέω, ἀπόστολος, ὀνομάζω, ἀποστέλλω, κηρύσσ...",2,0
387,tlg0031.tlg003.perseus-grc2.xml,Luke-Acts,Gospel of Luke,19458,tlg0031luke,tlg0031.tlg003,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],ΕΠΕΙΔΗΠΕΡ ΠΟΛΛΟΙ ἐπεχείρησαν ἀνατάξασθαι διήγ...,1274,"[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη...",[],0,"[[γίγνομαι, ἡμέρα, προσφωνέω, μαθητής, ἐκλεξάμ...",6,0
388,tlg0031.tlg004.perseus-grc2.xml,Johnannine literature (New Testament),Gospel of John,15590,tlg0031john,tlg0031.tlg004,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],"ΕΝ ΑΡΧΗ ἦν ὁ λόγος, καὶ ὁ λόγος ἦν πρὸς τὸν θ...",1164,"[[εἰμί, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, ...",[],0,"[[λέγω, εἰμί, δοῦλος, μέγας, κύριος, ἀπόστολος...",1,0
389,tlg0031.tlg005.perseus-grc2.xml,Luke-Acts,Acts,18406,tlg0031luke,tlg0031.tlg005,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],"τὸν μὲν πρῶτον λόγον ἐποιησάμην περὶ πάντων, ...",960,"[[πρῶτος, λόγος, ποιέω, πᾶς, Θεόφιλος, ἄρχω, Ἰ...","[[διέρχομαι, ὅλος, νῆσος, Πάφος, εὑρίσκω, ἀνήρ...",125,"[[πρῶτος, λόγος, ποιέω, πᾶς, Θεόφιλος, ἄρχω, Ἰ...",26,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417,tlg4081.tlg001.perseus-grc1.xml,Colluthus,Rape of Helen,2336,tlg4081,tlg4081.tlg001,A.D. 5,4.5,{'4.5': 1},NaN,christian,Epici/-ae,"νύμφαι Τρωιάδες, ποταμοῦ Ξάνθοιο γενέθλη, αἳ π...",209,"[[νύμφη, Τρῳάς, ποταμός, Ξάνθος, γενέθλη, πλόκ...",[],0,[],0,0
1418,tlg4084.tlg001.opp-grc1.xml,Zosimus,Historia Nova,62236,tlg4084,tlg4084.tlg001,A.D. 5?,4.5,{'4.5': 1},4.5,christian,Historici/-ae,"Πολυβίῳ τῷ Μεγαλοπολίτῃ, μνήμῃ παραδοῦναι τὰ ...",2265,"[[πολυβίῳ, μεγαλοπολίτῃ, μνήμη, παραδίδωμι, ἀξ...",[],0,[],0,0
1421,tlg4089.tlg003.opp-grc1.xml,Theodoret,Historia ecclesiastica,108815,tlg4089,tlg4089.tlg003,A.D. 4-5,4.5,"{'3.5': 0.5, '4.5': 0.5}",4.5,christian,Theologici,Τάδε ἔνεστιν ἐν τῷ πρώτῳ τόμῳ τῆς Θεοδωρήτου ἐ...,13146,"[[ἔνειμι, πρῶτος, τόμῳ, θεοδωρήτου, ἐκκλησιαστ...","[[προτρεπόμενον, Παῦλος, οὗτος, βιάζω, Πέτρος,...",31,"[[εὐσεβίος, παλαιστῖνος, ἱερόν, ἀπόστολος, ἱστ...",23,33
1422,tlg4089.tlg004.1st1K-grc1.xml,Theodoret,Historia Religiosa,44518,tlg4089,tlg4089.tlg004,A.D. 4-5,4.5,"{'3.5': 0.5, '4.5': 0.5}",4.5,christian,Theologici,"Τῶν ἀρίστων ἀνδρῶν, καὶ τῆς ἀρετῆς ἀθλητῶν, κ...",2948,"[[ἄριστος, ἀνήρ, ἀρετή, ἀθλητής, καλός, ὁράω, ...","[[τοιοῦτος, παντευχία, φάλαγξ, στρατηγός, πρόμ...",9,"[[ῥηθησομένων, ἀπιστέω, εὔδηλος, μωσέως, Ἰησοῦ...",13,9


In [8]:
LAGTec_by_date = pd.DataFrame(LAGTec.groupby("date_avr").sum())
LAGTec_by_date.reset_index(inplace=True)

LAGTec_by_date["authors_N"] = LAGTec_by_date["date_avr"].apply(lambda x: len(LAGTec[LAGTec["date_avr"]==x]["author"].unique()))

LAGTec_by_date["works_N"] = LAGTec_by_date["date_avr"].apply(lambda x: (LAGTec["date_avr"]==x).sum())

LAGTec_by_date["paul_freq"] = LAGTec_by_date["paul_N"] / LAGTec_by_date["wordcount"]

LAGTec_by_date[["date_avr", "authors_N", "wordcount", "n_sentences", "paul_N", "paul_freq", "sentences_paul_N", "sentences_apostle_N"]]

,date_avr,authors_N,wordcount,n_sentences,paul_N,paul_freq,sentences_paul_N,sentences_apostle_N
0,0.5,13,151723,10108,160,0.001055,156,84
1,1.5,13,322401,41076,18,0.000056,18,92
2,2.5,4,1237867,129474,497,0.000401,485,644
3,3.5,9,1347705,117599,213,0.000158,205,690
4,4.5,10,507127,45061,184,0.000363,181,146


In [9]:
sents_1to3 = [sen for work in LAGTec[LAGTec["date_avr"]<3]["lemmatized_sentences"] for sen in work]
sents_4to5 = [sen for work in LAGTec[LAGTec["date_avr"]>3]["lemmatized_sentences"] for sen in work]

In [11]:
len(sents_1to3)

180658

In [13]:
print(sents_1to3[:3])

[['βίβλος', 'γένεσις', 'Ἰησοῦς', 'Χριστός', 'υἱός', 'Δαυίδ', 'υἱός', 'Ἀβραάμ'], ['Ἀβραάμ', 'γεννάω', 'Ἰσαάκ', 'Ἰσαάκ', 'γεννάω', 'Ἰακώβ', 'Ἰακώβ', 'γεννάω', 'Ἰούδας', 'ἀδελφός', 'Ἰούδας', 'γεννάω', 'Φάρες', 'Ζάρα', 'Θαμάρ', 'Φάρες', 'γεννάω', 'Ἑσρώμ', 'Ἑσρώμ', 'γεννάω', 'Ἀράμ', 'Ἀράμ', 'γεννάω', 'Ἀμιναδάβ', 'Ἀμιναδάβ', 'γεννάω', 'Ναασσών', 'Ναασσών', 'γεννάω', 'Σαλμών', 'Σαλμών', 'γεννάω', 'βοῦς', 'Ῥαχάβ', 'βοῦς', 'γεννάω', 'Ἰωβήδ', 'Ῥούθ', 'Ἰωβήδ', 'γεννάω', 'Ἰεσσαί', 'Ἰεσσαί', 'γεννάω', 'Δαυίδ', 'βασιλεύς'], ['Δαυίδ', 'γεννάω', 'Σολομών', 'Οὐρίας', 'Σολομών', 'γεννάω', 'Ῥοβοάμ', 'Ῥοβοάμ', 'γεννάω', 'Ἀβιά', 'Ἀβιά', 'γεννάω', 'Ἀσάφ', 'Ἀσάφ', 'γεννάω', 'Ἰωσαφάτ', 'Ἰωσαφάτ', 'γεννάω', 'Ἰωράμ', 'Ἰωράμ', 'γεννάω', 'Ὀζίας', 'Ὀζίας', 'γεννάω', 'Ἰωαθάμ', 'Ἰωαθάμ', 'γεννάω', 'ἄχας', 'ἄχας', 'γεννάω', 'Ἑζεκίας', 'Ἑζεκίας', 'γεννάω', 'Μανασσῆς', 'Μανασσῆς', 'γεννάω', 'Ἀμώς', 'Ἀμώς', 'γεννάω', 'Ἰωσίας', 'Ἰωσίας', 'γεννάω', 'Ἰεχονίας', 'ἀδελφός', 'μετοικεσία', 'Βαβυλών']]


In [15]:
min_freq = 5

def get_vocab(docs, min_freq=5):
    words_flat = [item for sublist in docs for item in sublist]
    word_freq_tups = nltk.FreqDist(words_flat).most_common()
    vocabulary = [tup[0] for tup in word_freq_tups if tup[1] >= min_freq]
    #vocab_freqs = [len([doc for doc in docs if word in doc]) for word in vocabulary]
    return words_flat, vocabulary # , vocab_freqs

In [19]:
words_flat_1to3, vocabulary_1to3 = get_vocab(sents_1to3)
words_flat_4to5, vocabulary_4to5 = get_vocab(sents_4to5)

In [46]:
print("Παῦλος" in vocabulary_1to3)
print("Παῦλος" in vocabulary_4to5)
print("ἀπόστολος" in vocabulary_1to3)
print("ἀπόστολος" in vocabulary_4to5)

True
True
True
True


In [25]:
words_flat_1to3[-100:]

['ὄμμα',
 'μένω',
 'ἀπειλή',
 'σκώληξ',
 'σῶμα',
 'ἀπουσία',
 'ἐπιστρεφόμενον',
 'ἐκβράσαν',
 'σῶμα',
 'ἐπιστρεφής',
 'οὗτος',
 'ἐκφεύγω',
 'θεός',
 'εἰμί',
 'διδαχθείς',
 'ἔχω',
 'ἀθάνατος',
 'σῶμα',
 'ἄφθαρτ',
 'ψυχή',
 'βασιλεία',
 'οὐρανός',
 'ἀπόληψις',
 'γῆ',
 'βίος',
 'ἐπουράνιος',
 'βασιλεύς',
 'ἐπιγνούς',
 'εἰμί',
 'ὁμιλητής',
 'θεός',
 'συγκληρονόμος',
 'Χριστός',
 'ἐπιθυμία',
 'πάθος',
 'νόσος',
 'δουλούμενος',
 'γίγνομαι',
 'θεός',
 'ὑπομένω',
 'πάθος',
 'ἄνθρωπος',
 'εἰμί',
 'δίδωμι',
 'ἄνθρωπος',
 'εἷς',
 'ὅσος',
 'παρακολουθέω',
 'θεός',
 'παρέχω',
 'ἐπαγγέλλομαι',
 'θεός',
 'ἐθεοποιήθης',
 'ἀθάνατος',
 'γεννηθείς',
 'τουτέστι',
 'γιγνώσκω',
 'ἐπιγιγνώσκω',
 'πεποιηκότα',
 'θεός',
 'ἐπιγιγνώσκω',
 'ἐπιγνωσθῆναι',
 'συμβαίνω',
 'καλουμένῳ',
 'φιλεχθρήσητε',
 'ἄνθρωπος',
 'παλινδρομεῖν',
 'διστάσητε',
 'Χριστός',
 'πᾶς',
 'θεός',
 'ἁμαρτία',
 'ἄνθρωπος',
 'ἀποπλύνω',
 'προστάσσω',
 'νέος',
 'παλαιός',
 'ἄνθρωπος',
 'ἀποτελῶν',
 'εἰκών',
 'καλέω',
 'ἀρχή',
 'τύπος',
 'ἐπιδε

In [20]:
print(len(words_flat_1to3))
print(len(words_flat_4to5))

695239
810827


In [29]:
def get_cooc(docs, vocabulary=None, min_freq=5):
    if vocabulary==None:
        lemmata, vocabulary = get_vocab(docs, min_freq)
    vec_bow = CountVectorizer(vocabulary=vocabulary, lowercase=False)
    bow = vec_bow.fit_transform([" ".join(list(set(sentence))) for sentence in docs])
    cooc  = bow.T * bow
    cooc = cooc / len(docs)
    return cooc, vocabulary

In [30]:
cooc_1to3, vocabulary_1to3 = get_cooc(sents_1to3, vocabulary=vocabulary_1to3[:2000])
cooc_4to5, vocabulary_4to5 = get_cooc(sents_4to5, vocabulary=vocabulary_4to5[:2000])

In [33]:
cooc_4to5.todense().shape

(2000, 2000)

In [34]:
def normalize_ppmi3_matrix(pmi_matrix_df):
    minval, maxval = pmi_matrix_df.min().min(), pmi_matrix_df.max().max()
    minval_2 = minval * 2
    pmi_matrix_df.fillna(minval_2, inplace=True)
    pmi_matrix_norm_df = (pmi_matrix_df - minval_2) / (maxval - minval_2)
    return pmi_matrix_norm_df

In [35]:
def get_ppmi3_df(cooc, vocabulary, normalize=True, exp=3):
    pmi_rows_list = []
    for i in range(cooc.shape[1]):
        ab = np.array([row_el for row_el in list(cooc[i].toarray()[0])], dtype=float)
        ab_exp = np.power(ab, exp)
        axb = np.array([cooc[row_el[0], row_el[0]] * cooc[i, i] for row_el in enumerate(list(cooc[i].toarray()[0]))], dtype=float)
        pmi_row = np.divide(ab_exp, axb, out=np.zeros_like(ab_exp), where=axb!=0)
        pmi_row = [np.log(n) if n>0 else None for n in pmi_row]
        pmi_rows_list.append(pmi_row)
    pmi_matrix_df = pd.DataFrame(pmi_rows_list, columns=vocabulary, index=vocabulary)
    if normalize == True:
        pmi_matrix_df = normalize_ppmi3_matrix(pmi_matrix_df)
        np.fill_diagonal(pmi_matrix_df.to_numpy(), 1)
    return pmi_matrix_df #pmi_matrix_norm_df

In [40]:
def svd_reduction(cooc_matrix, n_components=300, random_state=1, n_iter=100):
    svd = TruncatedSVD(n_components=n_components, random_state=random_state, n_iter=n_iter)
    svd_matrix = svd.fit_transform(cooc_matrix)
    return svd_matrix

In [36]:
def from_docs_to_embeddings(docs, vocabulary=None, min_freq=5):
    cooc, vocabulary = get_cooc(docs, vocabulary=vocabulary, min_freq=min_freq)
    pmi_matrix = get_ppmi3_df(cooc, vocabulary)
    word_vectors_array = svd_reduction(pmi_matrix, n_components=300, random_state=1, n_iter=100)
    word_vectors_df = pd.DataFrame(word_vectors_array, index=vocabulary)
    pmi_svd_cos = pd.DataFrame(cosine_similarity(word_vectors_array), columns=vocabulary, index=vocabulary)
    return [cooc, vocabulary, pmi_matrix, word_vectors_df, pmi_svd_cos]

In [41]:
%%time
data_1to3 = from_docs_to_embeddings(sents_1to3, vocabulary=vocabulary_1to3[:2000])
data_4to5 = from_docs_to_embeddings(sents_4to5, vocabulary=vocabulary_4to5[:2000])

CPU times: user 14min 53s, sys: 1min 22s, total: 16min 15s
Wall time: 4min 10s


In [42]:
data_1to3[-1]

,εἰμί,λέγω,θεός,οὗτος,γίγνομαι,λόγος,πᾶς,ἄνθρωπος,αὐτός,ἔχω,...,χολή,συγγένεια,ἐπιφαίνω,χρίω,λίμνη,ἄνοια,σιγάω,ἀναλίσκω,ὀκτώ,κόρος
εἰμί,1.000000,0.999313,0.999386,0.999617,0.998827,0.997632,0.998620,0.996365,0.998418,0.997934,...,0.383185,0.420720,0.457354,0.409016,0.405139,0.406426,0.386402,0.393730,0.364559,0.404929
λέγω,0.999313,1.000000,0.998967,0.999342,0.998454,0.997188,0.998049,0.996010,0.998205,0.997439,...,0.387235,0.421316,0.459356,0.411176,0.405083,0.403971,0.391618,0.394767,0.365446,0.402210
θεός,0.999386,0.998967,1.000000,0.999245,0.998687,0.997445,0.998866,0.996580,0.998001,0.997516,...,0.381975,0.421682,0.461338,0.410029,0.403537,0.406821,0.388188,0.394757,0.362495,0.403034
οὗτος,0.999617,0.999342,0.999245,1.000000,0.998677,0.997956,0.998533,0.996424,0.998369,0.997942,...,0.384384,0.421604,0.458724,0.409195,0.404079,0.406907,0.387492,0.393033,0.365072,0.405355
γίγνομαι,0.998827,0.998454,0.998687,0.998677,1.000000,0.997151,0.998048,0.996145,0.997333,0.996988,...,0.384454,0.424337,0.464407,0.413830,0.410775,0.409977,0.391502,0.399161,0.370126,0.403365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ἄνοια,0.406426,0.403971,0.406821,0.406907,0.409977,0.411959,0.413806,0.415020,0.411761,0.407156,...,0.447134,0.481516,0.464135,0.412739,0.429618,1.000000,0.414957,0.409578,0.419386,0.551645
σιγάω,0.386402,0.391618,0.388188,0.387492,0.391502,0.396068,0.393815,0.395306,0.393829,0.389796,...,0.416101,0.496000,0.504560,0.455825,0.445250,0.414957,1.000000,0.456409,0.449851,0.369757
ἀναλίσκω,0.393730,0.394767,0.394757,0.393033,0.399161,0.398144,0.401389,0.403412,0.398616,0.397577,...,0.496596,0.465703,0.428387,0.438772,0.373511,0.409578,0.456409,1.000000,0.358671,0.391451
ὀκτώ,0.364559,0.365446,0.362495,0.365072,0.370126,0.367344,0.366630,0.365524,0.366993,0.364021,...,0.391042,0.495934,0.362624,0.416816,0.389343,0.419386,0.449851,0.358671,1.000000,0.356181


In [49]:
pickle.dump(data_1to3, open("../data/large_files/embeddings_1to3.pkl", "wb"), pickle.HIGHEST_PROTOCOL)
pickle.dump(data_4to5, open("../data/large_files/embeddings_4to5.pkl", "wb"), pickle.HIGHEST_PROTOCOL)